In [1]:
# Have to patch sys.path until we have tighter taucmdr/anaconda integration.
# Need to move towards taucmdr as a conda package.
import os
import sys
try:
    import taucmdr
except ImportError:
    sys.path.insert(0, os.path.join(os.environ['__TAUCMDR_HOME__'], 'packages'))
finally:
    from taucmdr.model.project import Project

In [2]:
%pylab
import matplotlib
import matplotlib.pyplot as plt
matplotlib.style.use('ggplot')
import pandas as pd
import math
import numpy as np
import operator

Using matplotlib backend: Qt5Agg
Populating the interactive namespace from numpy and matplotlib


In [3]:
trials = Project.selected().experiment().trials()
trial0_data = trials[0].get_data()

In [4]:
trial0_intervals = trial0_data[0][0][0].interval_data()
trial0_intervals.plot(kind='bar')
plt.show()

In [5]:
trial0_intervals = {}
for i in xrange(0, len(trial0_data)):
    trial0_intervals[i] = {}
    for j in xrange(0, len(trial0_data[i])):
        trial0_intervals[i][j] = {}
        for k in xrange(0, len(trial0_data[i][j])):
            trial0_intervals[i][j][k] = {}
            trial0_intervals[i][j][k] = trial0_data[i][j][k].interval_data()

In [6]:
def get_regions(dfs):
    regions = set()
    for rank in dfs:
        for context in dfs[rank]:
            for thread in dfs[rank][context]:
                df = dfs[rank][context][thread]
                regions.update(df.index.tolist())
    return regions

def largest_stddev(dfs,n):
    # Use Welford's Method (see https://stackoverflow.com/questions/895929/how-do-i-determine-the-standard-deviation-stddev-of-a-set-of-values)
    regions = get_regions(dfs)
    stddevs = {}
    for region in regions:
        mean = 0.0
        stddev = 0.0
        k = 1
        for rank in dfs:
            for context in dfs[rank]:
                for thread in dfs[rank][context]:
                    tmpM = mean
                    exclusive_time = dfs[rank][context][thread].at[region,'Exclusive']
                    mean += (exclusive_time - tmpM) / k
                    stddev += (exclusive_time - tmpM) * (exclusive_time - mean)
                    k += 1

        stddev = math.sqrt(stddev/(k-1))
        stddevs[region] = stddev
        
    largest = [x[0] for x in sorted(stddevs.iteritems(), key=operator.itemgetter(1), reverse=True)[:n]]
    return largest


def largest_correlation(dfs,n):
    pass

def largest_exclusive(df,n):
    rows = df.nlargest(n, 'Exclusive')
    #return set(rows.index)
    return list(rows.index)

def hotspots(dfs, n, flag):
    if flag == 0:
        largest = largest_exclusive(dfs[0][0][0],n)
    elif flag == 1:
        largest = largest_stddev(dfs,n)
    elif flag == 2:
        largest = largest_correlation(dfs,n)
    else:
        print 'Invalid flag'
    y = ['exclusive time', 'standard deviation', 'correlation to total runtime']
    print 'Hotspot Analysis Summary'
    print '='*80
    print 'The code regions with largest %s are: ' %y[flag]
    for i in xrange(0,n):
        print '%s: %s' %(i+1, largest[i])

hotspots(trial0_intervals, 5, 0)
hotspots(trial0_intervals, 5, 1)

Hotspot Analysis Summary
The code regions with largest exclusive time are: 
1: void compute(double **, double **, double **, int, int, int) C [{matmult.c} {90,1}-{109,1}]  
2: .TAU application => int main(int, char **) C [{matmult.c} {209,1}-{299,1}]   => double do_work(void) C [{matmult.c} {132,1}-{168,1}]   => void compute(double **, double **, double **, int, int, int) C [{matmult.c} {90,1}-{109,1}]  
3: .TAU application => int main(int, char **) C [{matmult.c} {209,1}-{299,1}]   => double do_work(void) C [{matmult.c} {132,1}-{168,1}]   => void compute_interchange(double **, double **, double **, int, int, int) C [{matmult.c} {111,1}-{130,1}]  
4: void compute_interchange(double **, double **, double **, int, int, int) C [{matmult.c} {111,1}-{130,1}]  
5: MPI_Finalize()  
Hotspot Analysis Summary
The code regions with largest standard deviation are: 
1: .TAU application => int main(int, char **) C [{matmult.c} {209,1}-{299,1}]   => MPI_Finalize()  
2: MPI_Finalize()  
3: void comput